In [1]:
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
xr.__version__

'0.11.1+69.gc770eec'

In [2]:
import zarr
zarr.__version__

'2.2.1.dev146'

In [3]:
import intake
list(intake.cat)

['cmip6_monthly', 'cmip6_3hourly']

In [4]:
df = intake.cat.cmip6_3hourly.read()
list(df.source_id.unique())

['IPSL-CM6A-LR', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'GISS-E2-1-G', 'BCC-CSM2-MR']

In [5]:
# drop the 365_day models for now
df = df.drop(df.loc[df['source_id'] == 'BCC-CSM2-MR'].index)
df = df.drop(df.loc[df['source_id'] == 'GISS-E2-1-G'].index)

# use the local files instead of the OPENDAP_url in aggregation
df['OPENDAP_url'] = df.file_fullpath

In [6]:
models = list(df.source_id.unique())

In [7]:
files = df[df["variable_id"] == "pr"]
files.head()

,Unnamed: 0,File_Name,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version_id,file_fullpath,file_path,OPENDAP_url,HTTPServer_url
0,0,pr_3hr_IPSL-CM6A-LR_historical_r7i1p1f1_gr_185...,CMIP,IPSL,IPSL-CM6A-LR,historical,r7i1p1f1,3hr,pr,gr,v20180803,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,CMIP/IPSL/IPSL-CM6A-LR/historical/r7i1p1f1/3hr...,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,http://mary.ldeo.columbia.edu/testing/CMIP6hr3...
1,1,pr_3hr_IPSL-CM6A-LR_historical_r7i1p1f1_gr_195...,CMIP,IPSL,IPSL-CM6A-LR,historical,r7i1p1f1,3hr,pr,gr,v20180803,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,CMIP/IPSL/IPSL-CM6A-LR/historical/r7i1p1f1/3hr...,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,http://mary.ldeo.columbia.edu/testing/CMIP6hr3...
6,6,pr_3hr_IPSL-CM6A-LR_historical_r6i1p1f1_gr_195...,CMIP,IPSL,IPSL-CM6A-LR,historical,r6i1p1f1,3hr,pr,gr,v20180803,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,CMIP/IPSL/IPSL-CM6A-LR/historical/r6i1p1f1/3hr...,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,http://mary.ldeo.columbia.edu/testing/CMIP6hr3...
7,7,pr_3hr_IPSL-CM6A-LR_historical_r6i1p1f1_gr_185...,CMIP,IPSL,IPSL-CM6A-LR,historical,r6i1p1f1,3hr,pr,gr,v20180803,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,CMIP/IPSL/IPSL-CM6A-LR/historical/r6i1p1f1/3hr...,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,http://mary.ldeo.columbia.edu/testing/CMIP6hr3...
12,12,pr_3hr_IPSL-CM6A-LR_historical_r3i1p1f1_gr_195...,CMIP,IPSL,IPSL-CM6A-LR,historical,r3i1p1f1,3hr,pr,gr,v20180803,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,CMIP/IPSL/IPSL-CM6A-LR/historical/r3i1p1f1/3hr...,/dm3/naomi/CMIP6hr3/CMIP/IPSL/IPSL-CM6A-LR/his...,http://mary.ldeo.columbia.edu/testing/CMIP6hr3...


In [8]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:34299 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 40 Memory: 134.90 GB


In [9]:
%%time
from esgf2zarr import aggregate

zarrdir = 'zarr-test-consolidated/'

dsets = aggregate.combine_files(files)

/usr/local/python/anaconda3/envs/pangeo3.7/lib/python3.6/site-packages/tqdm-4.31.1-py3.6.egg/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)



CPU times: user 7min 39s, sys: 1min 11s, total: 8min 50s
Wall time: 1h 9min 7s


In [10]:
names = list(dsets.keys())
names

['CNRM-CERFACS.CNRM-CM6-1.historical.3hr.gr',
 'CNRM-CERFACS.CNRM-ESM2-1.historical.3hr.gr',
 'IPSL.IPSL-CM6A-LR.historical.3hr.gr']

In [11]:
for name in names:
    nt = dsets[name].time.size
    dsets[name]['time'] = pd.date_range('1850-01-01T01:30', periods=nt, freq='3H')

In [12]:
import os
from dask.diagnostics import ProgressBar

for i, name in enumerate(names):
    zdir = zarrdir +  name
    command = "rm -rf " + zdir
    os.system(command)
    with ProgressBar():
        print(name,zdir)
        dsets[name].to_zarr(zdir,consolidated=True)

CNRM-CERFACS.CNRM-CM6-1.historical.3hr.gr zarr-test-consolidated/CNRM-CERFACS.CNRM-CM6-1.historical.3hr.gr
CNRM-CERFACS.CNRM-ESM2-1.historical.3hr.gr zarr-test-consolidated/CNRM-CERFACS.CNRM-ESM2-1.historical.3hr.gr
IPSL.IPSL-CM6A-LR.historical.3hr.gr zarr-test-consolidated/IPSL.IPSL-CM6A-LR.historical.3hr.gr


In [13]:
! du -sh zarr-test-consolidated

362G	zarr-test-consolidated


In [14]:
# test one of the datasets

zarrdir = '/dm3/naomi/zarr-test-consolidated/'

dss = xr.open_zarr(zarrdir+'CNRM-CERFACS.CNRM-CM6-1.historical.3hr.gr')
dss

<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lat: 128, lon: 256, member_id: 2, time: 482120)
Coordinates:
  * lat          (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon          (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
  * member_id    (member_id) object 'r1i1p1f2' 'r2i1p1f2'
  * time         (time) datetime64[ns] 1850-01-01T01:30:00 ... 2014-12-31T22:30:00
    time_bounds  (time, axis_nbounds) float64 dask.array<shape=(482120, 2), chunksize=(482120, 2)>
Dimensions without coordinates: axis_nbounds
Data variables:
    pr           (member_id, time, lat, lon) float32 dask.array<shape=(2, 482120, 128, 256), chunksize=(1, 709, 128, 256)>
Attributes:
    CMIP6_CV_version:      cv=6.2.3.0-7-g2019642
    Conventions:           CF-1.7 CMIP-6.2
    NCO:                   "4.5.5"
    activity_id:           CMIP
    arpege_minor_version:  6.3.2
    branch_method:         standard
    branch_time_in_child:  0.0
    contact:               con